In [304]:
using JuAD
using Zygote

In [305]:
function zygote_gradient(f, args...)
    return jacobian((args...) -> f(args...), args...)
end

zygote_gradient (generic function with 1 method)

In [306]:
A = [2; 8]
X = [10 2]
b = [2; 3]

fZ(A, X, b) = sum((A * X) .+ b)
println(zygote_gradient(fZ, A, X, b))

fAD(A, X, b) = sum((Variable(A, name="A") * Variable(X, name="X")) .+ Variable(b))
order = topological_sort(fAD(A, X, b))
y = forward!(order)
backward!(order)

println(order[1].∇)
println(order[2].∇)
println(order[4].∇)

([12 12], [10 10], [2 2])
Float32[12.0; 12.0;;]
Float32[10.0 10.0]
Float32[1.0 1.0; 1.0 1.0]


In [307]:
A = [11; 212]
X = [10 2]

fZ(A, X) = sum(A*X) + 100
println(zygote_gradient(fZ, A, X))

fAD(A, X) = sum(Variable(A, name="A") * Variable(X, name="X")) .+ Constant(100)
order = topological_sort(fAD(A, X))
y = forward!(order)
backward!(order)

println(order[1].∇)
println(order[2].∇)

([12 12], [223 223])
Float32[12.0; 12.0;;]
Float32[223.0 223.0]


In [308]:
A = [11; 12]
X = [10 2]
b = [2; 2]
c = [7; 7]

fZ2(A, x, b, c) = sum((A*x ./ b) .- c)
println(zygote_gradient(fZ2, A, X, b, c))

fAD2(A, X, b, c) = sum((Variable(A) * Variable(X) ./ Variable(b)) .- Variable(c)) 
order = topological_sort(fAD2(A, X, b, c))
y = forward!(order)
backward!(order) 

println(order[1].∇)
println(order[2].∇)
println(order[4].∇)
println(order[6].∇)


([6.0 6.0], [11.5 11.5], [-33.0 -36.0], [-2.0 -2.0])
Float32[6.0; 6.0;;]
Float32[11.5 11.5]
Float32[-27.5 -5.5; -30.0 -6.0]
Float32[-1.0 -1.0; -1.0 -1.0]


In [309]:
A = [11; 12]
X = [10 2]

function relu(x)
    max.(0.0f0, x)
end


fZ3(A, X) = sum(relu(A*X))
println(zygote_gradient(fZ3, A, X))

fAD3(A, X) = sum(ReLU(Variable(A, name="A") * Variable(X, name="X")))
order = topological_sort(fAD3(A, X))
y = forward!(order)
backward!(order)

println(order[1].∇)
println(order[2].∇)

(Float32[12.0 12.0], Float32[23.0 23.0])
Float32[12.0; 12.0;;]
Float32[23.0 23.0]
